In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
input_size = 784       # The image size = 28 x 28 = 784
hidden_size = 500      # The number of nodes at the hidden layer
num_classes = 10       # The number of output classes. In this case, from 0 to 9
num_epochs = 5         # The number of times entire dataset is trained
batch_size = 100       # The size of input data took for one iteration
learning_rate = 0.001  # The speed of convergence

In [3]:
train_dataset = dsets.MNIST(root='./data',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

In [4]:

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [5]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer: 784 (input data) -> 500 (hidden node)
        self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
        self.fc2 = nn.Linear(hidden_size, num_classes) # 2nd Full-Connected Layer: 500 (hidden node) -> 10 (output class)
    
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [6]:
net = Net(input_size, hidden_size, num_classes)


In [7]:
# net.cuda()    # You can comment out this line to disable GPU

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [12]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):   # Load a batch of images with its (index, data, class)
        images = Variable(images.view(-1, 28*28))         # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
        labels = Variable(labels)
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = net(images)                             # Forward pass: compute the output class given a image
        loss = criterion(outputs, labels)                 # Compute the loss: difference between the output class and the pre-given label
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        
        if (i+1) % 100 == 0:                              # Logging
#             print("SMH")
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data.item()))
            

Epoch [1/5], Step [100/600], Loss: 0.0169
Epoch [1/5], Step [200/600], Loss: 0.0100
Epoch [1/5], Step [300/600], Loss: 0.0146
Epoch [1/5], Step [400/600], Loss: 0.0222
Epoch [1/5], Step [500/600], Loss: 0.0326
Epoch [1/5], Step [600/600], Loss: 0.0478
Epoch [2/5], Step [100/600], Loss: 0.0124
Epoch [2/5], Step [200/600], Loss: 0.0260
Epoch [2/5], Step [300/600], Loss: 0.0161
Epoch [2/5], Step [400/600], Loss: 0.0252
Epoch [2/5], Step [500/600], Loss: 0.0158
Epoch [2/5], Step [600/600], Loss: 0.0145
Epoch [3/5], Step [100/600], Loss: 0.0129
Epoch [3/5], Step [200/600], Loss: 0.0069
Epoch [3/5], Step [300/600], Loss: 0.0099
Epoch [3/5], Step [400/600], Loss: 0.0373
Epoch [3/5], Step [500/600], Loss: 0.0072
Epoch [3/5], Step [600/600], Loss: 0.0102
Epoch [4/5], Step [100/600], Loss: 0.0180
Epoch [4/5], Step [200/600], Loss: 0.0182
Epoch [4/5], Step [300/600], Loss: 0.0071
Epoch [4/5], Step [400/600], Loss: 0.0040
Epoch [4/5], Step [500/600], Loss: 0.0236
Epoch [4/5], Step [600/600], Loss:

In [13]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
    total += labels.size(0)                    # Increment the total count
    correct += (predicted == labels).sum()     # Increment the correct count
    
print('Accuracy of the network on the 10K test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10K test images: 97 %
